In [25]:
import uproot
import matplotlib.pyplot as plt
import seaborn
import numpy as np
import math
from scipy.fft import fft, fftfreq
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import os
from plotly.subplots import make_subplots
from plotly import tools
import plotly.offline as pyo
import sys
from tqdm import tqdm

In [40]:
#Opens log files that contain interaction information
#with open('/Users/danielcarber/Documents/ICARUS/NuMI_Nue/1e1p/output_nu.log') as file:
#    lines_nu = [line_nu.rstrip() for line_nu in file]
with open('/Users/danielcarber/Documents/ICARUS/NuMI_Nue/1e1p/output_nu_all_planes.log') as file:
    lines_nu = [line_nu.rstrip() for line_nu in file]
#with open('/Users/danielcarber/Documents/ICARUS/NuMI_Nue/1e1p/output_data.log') as file:
#    lines_data = [line_data.rstrip() for line_data in file]
print(lines_nu[1])

SIGNAL,1,3,0,0,1.000000,0.000000,2.000000,2.000000,0.000000,659.956883,570.640325,244.848184,246.042173,1,1,4,4,596.992192,853.398284,531.496033,477.313660,692.281311,697.702576,0.935215,0.989412,1.185676,1.178460,1.779036,1.807567,513.492986,679.090106,-nan,-nan,1.580879,1.810575,0.904355,0.997297,1,1,1,


In [41]:
#Dictionaries to be filled in interaction information
#Signal is true interactions matched with reconstructed intercations
df_signal_nu = {'Type': [], 'Run': [], 'Evt': [], 'Subrun':[],'Nu_id':[],'Image_id':[],'Interaction_id':[], 'Category':[],
     'Category_topology':[],'category_interaction_mode':[],'leading_electron_ke_true':[], 'leading_electron_ke_reco':[],
     'leading_proton_ke_true':[],'leading_proton_ke_reco':[],'leading_electron_pid_true':[], 'leading_electron_pid_reco':[],
     'leading_proton_pid_true':[],'leading_proton_pid_reco':[],'visible_energy_true':[],'visible_energy_reco':[],'leading_electron_pt_true':[],
     'leading_electron_pt_reco':[],'leading_proton_pt_true':[],'leading_proton_pt_reco':[],'electron_angle_polar_angle_true':[],
     'leading_electron_polar_angle_reco':[],'electron_azimuthal_angle_true':[],'electron_azimuthal_angle_reco':[],
     'opening_angle_true':[],'opening_angle_reco':[],'interaction_pt_true':[],'interaction_pt_reco':[],
     'phiT_true':[],'phiT_reco':[],'alphaT_true':[],'alphaT_reco':[],'electron_softmax':[],'proton_softmax':[],
     'all_1e1p_cut':[],'all_1eNp_cut':[],'all_1eX_cut':[],'End':[]}

#Selected is reco interactions matched with true interactions
df_selected_nu = df_signal_nu
list_df_MC = ['Type', 'Run', 'Evt', 'Subrun','Nu_id','Image_id','Interaction_id', 'Category',
     'Category_topology','category_interaction_mode','leading_electron_ke_true', 'leading_electron_ke_reco',
     'leading_proton_ke_true','leading_proton_ke_reco','leading_electron_pid_true', 'leading_electron_pid_reco',
     'leading_proton_pid_true','leading_proton_pid_reco','visible_energy_true','visible_energy_reco','leading_electron_pt_true',
     'leading_electron_pt_reco','leading_proton_pt_true','leading_proton_pt_reco','electron_angle_polar_angle_true',
     'leading_electron_polar_angle_reco','electron_azimuthal_angle_true','electron_azimuthal_angle_reco',
     'opening_angle_true','opening_angle_reco','interaction_pt_true','interaction_pt_reco',
     'phiT_true','phiT_reco','alphaT_true','alphaT_reco','electron_softmax','proton_softmax','all_1e1p_cut','all_1eNp_cut','all_1eX_cut',"End"]


#Fills the dictionaries with data            
for line_nu in lines_nu:
    line_nu = line_nu.split(",")
    if line_nu[0] == 'SIGNAL':
        for i in range(len(line_nu)):
            df_signal_nu[list_df_MC[i]].append(line_nu[i])
    elif line_nu[0] == 'SELECTED':
        for i in range(len(line_nu)):
            df_selected_nu[list_df_MC[i]].append(line_nu[i])

In [42]:
signal,sel_1e, sel_1e1pi1p, sel_nue_other, sel_NC, sel_cosmic,sel_numu,sel_other = [],[],[],[],[],[],[],[]
total_events =0
for index, category in enumerate(df_selected_nu['Category_topology']):
    if df_selected_nu['all_1e1p_cut'][index] == '1' or df_selected_nu['all_1eNp_cut'][index] == '1':
        print(df_selected_nu['visible_energy_reco'][index])
        total_events +=1
        if (category[0] == '0' or category[0] == '2'):
            signal.append(float(df_selected_nu['visible_energy_reco'][index]))
        elif category[0] == '1':
            sel_nue_other.append(float(df_selected_nu['visible_energy_reco'][index]))
        elif category[0] == '3':
            sel_nue_other.append(float(df_selected_nu['visible_energy_reco'][index]))
        elif category[0] == '4':
            sel_nue_other.append(float(df_selected_nu['visible_energy_reco'][index]))
        elif category[0] == '5':
            sel_NC.append(float(df_selected_nu['visible_energy_reco'][index]))
        elif category[0] == '6':
            sel_cosmic.append(float(df_selected_nu['visible_energy_reco'][index]))
        elif category[0] == '7':
            sel_numu.append(float(df_selected_nu['visible_energy_reco'][index]))
        else:
            print()
            sel_other.append(float(df_selected_nu['visible_energy_reco'][index]))

594.911656
853.398284
853.398284
794.740949
451.141493
451.141493
585.681774
585.681774
470.927630
470.927630
793.459994
379.403907
1552.578983
1552.578983
565.591916
565.591916
792.750648
980.805997
980.805997
541.391662
541.391662
992.023661
992.023661
753.138455
753.138455
870.285848
870.285848
1175.723839
591.936086
591.936086
510.430443
510.430443
1401.763841
1401.763841
300.004269
1129.053255
1129.053255
164.187488
1032.337356
1032.337356
620.000080
3654.686305
3654.686305
1642.502423
1642.502423
2134.398607
2134.398607
1195.132284
1195.132284
381.365132
381.365132
928.681165
928.681165
788.098169
788.098169
598.010927
598.010927
437.879662
437.879662
1012.287585
817.700697
1825.468918
1825.468918
2848.862018
2848.862018
716.864137
716.864137
1482.774512
1482.774512
291.294523
291.294523
909.910538
909.910538
667.480097
667.480097
427.244362
427.244362
1226.439251
1226.439251
1154.798669
932.532416
932.532416
1460.833630
1460.833630
764.773218
764.773218
601.089761
1102.299881
11

In [43]:
def hist_1d(metric, bounds,bins, xaxis_name, Plot_title, selection):
    fig = make_subplots(rows=1,cols=1,subplot_titles = (f"<span style='font-size: 42px;'>{Plot_title}",))
    
    
    print('Purity: ', len(signal)/total_events)
    print('Number of signal: ',len(signal),' Number of selected:', total_events)
    cosmic_size = len(sel_cosmic)
    numu_size = len(sel_numu)
    NC_size = len(sel_NC)
    signal_size = len(signal)
    nue_size = len(sel_nue_other)
    other_size = len(sel_other)
    
    
    fig.add_trace(go.Histogram(x=sel_cosmic,marker_color = 'yellow',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                               name = f'Cosmics ({cosmic_size}, {100*cosmic_size/total_events:.2f}%)'),row = 1, col =1)
    fig.add_trace(go.Histogram(x=sel_numu,marker_color = 'blue',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                              name = f'\u03BD<sub>\u03BC</sub> ({numu_size}, {100*numu_size/total_events:.2f}%)'),row = 1, col =1)
    fig.add_trace(go.Histogram(x=sel_NC,marker_color = 'black',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                              name = f'NC ({NC_size}, {100*NC_size/total_events:.2f}%)'),row = 1, col =1)
    fig.add_trace(go.Histogram(x=sel_other,marker_color = 'purple',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                              name = f'Other ({other_size}, {100*other_size/total_events:.2f}%)'),row = 1, col =1)
    
    if selection != 'Nue':
        #nue_size =  len(df[metric][mask_other_nue][reco_signal][mask])
        fig.add_trace(go.Histogram(x=sel_nue_other,marker_color = 'orange',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                                  name = f'Other \u03BD<sub>e</sub> ({nue_size}, {100*nue_size/total_events:.2f}%)'),row = 1, col =1)
        fig.add_trace(go.Histogram(x=signal,marker_color = 'darkgreen',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                              name = f'{selection} ({signal_size}, {100*signal_size/total_events:.2f}%)'),row = 1, col =1)
    else:
        fig.add_trace(go.Histogram(x=signal,marker_color = 'darkgreen',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                              name = f'\u03BD<sub>e</sub> ({signal_size}, {100*signal_size/total_events:.2f}%)'),row = 1, col =1)
    
    
    fig.update_xaxes(title_text = xaxis_name,row = 1, col = 1)
    fig.update_yaxes(title_text = "Entries",row = 1, col = 1,)
    fig.update_layout(font = dict(size=26),legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.43,
        bgcolor="LightSteelBlue",
            bordercolor="Black",
    ))
    fig.update_annotations(font_size=46)
    fig.update_layout(barmode='stack')
    fig.update_layout(xaxis = dict(range = bounds))
    fig.update_layout(height = 900, width = 900,showlegend = True)
    fig.show()


In [44]:
#hist_1d('reco_nu_energy',[0,5000],[0,5000,250],'Reco Neutrino Energy [MeV]','Selected 1e1p Events','1e1p')
hist_1d('reco_nu_energy',[0,5000],[0,5000,250],'Reco Neutrino Energy [MeV]','Selected 1eNp Events','1eNp')
#hist_1d('reco_nu_energy',[0,5000],[0,5000,100],'Reco Neutrino Energy [MeV]','Selected CC \u03BD<sub>e</sub> Inclusive Events','Nue')

Purity:  0.7452093683463449
Number of signal:  2100  Number of selected: 2818
